In [35]:
import pandas as pd
import numpy as np
import snowflake.connector
from snowflake.snowpark import Session
from snowflake.ml.utils import connection_params

In [29]:
USERNAME = '<>'
PASSWORD = '<>'
ACCOUNT = '<>' # go to Admin page and extract from the link
ROLE = 'ACCOUNTADMIN'
WAREHOUSE = 'CLIMATOLOGY'
DATABASE = 'ML_CLIMATOLOGY_DB'
SCHEMA = 'ML_CLIMATOLOGY_SCHEMA'

In [6]:
connection_parameters = {
    "user": USERNAME,
    "password": PASSWORD,
    "account": ACCOUNT,
    "role": ROLE,
    "warehouse": WAREHOUSE,
    "database": DATABASE,
    "schema": SCHEMA,
    }
session = Session.builder.configs(connection_parameters).create()

In [30]:
conn = snowflake.connector.connect(
    user=USERNAME,
    password=PASSWORD,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=SCHEMA
)

# Create a cursor object
cursor = conn.cursor()

In [31]:
# sql = "select * from GLOBAL_WEATHER__CLIMATE_DATA_FOR_BI.STANDARD_TILE.CLIMATOLOGY_DAY"

sql1 = 'USE ROLE ACCOUNTADMIN;'
sql2 = 'CREATE OR REPLACE WAREHOUSE CLIMATOLOGY;'
sql3 = 'CREATE OR REPLACE DATABASE ML_CLIMATOLOGY_DB;'
sql4 = 'CREATE OR REPLACE SCHEMA ML_CLIMATOLOGY_SCHEMA;'
sql5 = 'CREATE OR REPLACE STAGE ML_CLIMATOLOGY_ASSETS;'
sql6 = '''CREATE FILE FORMAT IF NOT EXISTS ML_CLIMATOLOGY_DB.ML_CLIMATOLOGY_SCHEMA.JSONFORMAT 
    TYPE = "JSON";'''
sql7 = '''CREATE STAGE IF NOT EXISTS ML_CLIMATOLOGY_DB.ML_CLIMATOLOGY_SCHEMA.CLIMATOLOGY 
    FILE_FORMAT = ML_CLIMATOLOGY_DB.ML_CLIMATOLOGY_SCHEMA.JSONFORMAT  
    URL = "s3://snowflake-climatology/cli_total.json"'''
# df = session.sql(sql)

In [32]:
cursor.execute(sql1)
cursor.execute(sql2)
cursor.execute(sql3)
cursor.execute(sql4)
cursor.execute(sql5)
cursor.execute(sql6)
cursor.execute(sql7)


In [27]:
session.close()

In [34]:
cursor.close()

True

In [ ]:
# create or replace warehouse snowpark_opt_wh with
#   warehouse_size = 'MEDIUM'
#   warehouse_type = 'SNOWPARK-OPTIMIZED';
#   max_concurrency_level = 1;

# create or replace procedure train()
#   returns variant
#   language python
#   runtime_version = 3.8
#   packages = ('snowflake-snowpark-python', 'scikit-learn', 'joblib')
#   handler = 'main'
# as $$
# import os
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split, GridSearchCV
# from joblib import dump

# def main(session):
#   # Load features
#   df = session.table('MARKETING_BUDGETS_FEATURES').to_pandas()
#   X = df.drop('REVENUE', axis = 1)
#   y = df['REVENUE']

#   # Split dataset into training and test
#   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

#   # Preprocess numeric columns
#   numeric_features = ['SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL']
#   numeric_transformer = Pipeline(steps=[('poly',PolynomialFeatures(degree = 2)),('scaler', StandardScaler())])
#   preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features)])

#   # Create pipeline and train
#   pipeline = Pipeline(steps=[('preprocessor', preprocessor),('classifier', LinearRegression(n_jobs=-1))])
#   model = GridSearchCV(pipeline, param_grid={}, n_jobs=-1, cv=10)
#   model.fit(X_train, y_train)

#   # Upload trained model to a stage
#   model_file = os.path.join('/tmp', 'model.joblib')
#   dump(model, model_file)
#   session.file.put(model_file, "@ml_models",overwrite=True)

#   # Return model R2 score on train and test data
#   return {"R2 score on Train": model.score(X_train, y_train),"R2 score on Test": model.score(X_test, y_test)}
# $$;